In [1]:
# Установка необходимых зависимостей
!pip install ultralytics
!pip install torch torchvision
!pip install albumentations

# Настройка CUDA для лучшего управления памятью
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Клонирование репозитория с обученной моделью
!git clone https://github.com/smicurin474/Car_plate_detecting.git

# Добавляем путь к репозиторию в PYTHONPATH
import sys
sys.path.append('/kaggle/working/Car_plate_detecting')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 17.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninst

In [2]:
# Загрузка датасета
!mkdir -p /kaggle/working/finetune_dataset
!wget https://huggingface.co/datasets/AY000554/Car_plate_detecting_dataset/resolve/main/train.zip
!wget https://huggingface.co/datasets/AY000554/Car_plate_detecting_dataset/resolve/main/val.zip

# Распаковка датасета
!unzip -q train.zip -d /kaggle/working/finetune_dataset
!unzip -q val.zip -d /kaggle/working/finetune_dataset

--2025-09-03 15:52:22--  https://huggingface.co/datasets/AY000554/Car_plate_detecting_dataset/resolve/main/train.zip
Resolving huggingface.co (huggingface.co)... 18.244.202.68, 18.244.202.60, 18.244.202.73, ...
Connecting to huggingface.co (huggingface.co)|18.244.202.68|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/f5/fb/f5fb9d95f122eb06ad36fa568b80b423cda743737da1285f498b17999aee4770/689e09a59fd1ba57584e45896ae9d87a205a201080d23116362d41c5e1b805c3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27train.zip%3B+filename%3D%22train.zip%22%3B&response-content-type=application%2Fzip&Expires=1756918342&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1NjkxODM0Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2Y1L2ZiL2Y1ZmI5ZDk1ZjEyMmViMDZhZDM2ZmE1NjhiODBiNDIzY2RhNzQzNzM3ZGExMjg1ZjQ5OGIxNzk5OWFlZTQ3NzAvNjg5ZTA5YTU5ZmQxYmE1NzU4NGU0NTg5NmFlOWQ4N2EyMDVhMjAxMDgwZD

In [4]:
# Создаем отдельную директорию для тестирования аугментаций на сложных случаях
!mkdir -p /kaggle/working/hard_cases_test/images
!cp -r /kaggle/working/Car_plate_detecting/hard_cases/* /kaggle/working/hard_cases_test/images/

print("Количество изображений для дообучения:", len(os.listdir('/kaggle/working/finetune_dataset/train/images')))
print("Количество изображений для валидации:", len(os.listdir('/kaggle/working/finetune_dataset/val/images')))
print("Количество сложных случаев для тестирования:", len(os.listdir('/kaggle/working/hard_cases_test/images')))

Количество изображений для дообучения: 20505
Количество изображений для валидации: 2563
Количество сложных случаев для тестирования: 37


In [30]:
# Функции для анализа сложных случаев
from pathlib import Path
import cv2
import numpy as np
import matplotlib.pyplot as plt
import yaml
from scipy.stats import entropy
from skimage.filters import threshold_otsu
from skimage.feature import local_binary_pattern

def compute_image_quality_metrics(image):
    """Расчет метрик качества изображения"""
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image
        
    # Метрики резкости
    laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
    gradients = np.gradient(gray.astype(float))
    gradient_magnitude = np.sqrt(gradients[0]**2 + gradients[1]**2)
    gradient_mean = np.mean(gradient_magnitude)
    
    # Метрики контраста
    contrast = gray.std()
    local_contrast = np.mean([np.std(gray[i:i+16, j:j+16]) 
                            for i in range(0, gray.shape[0]-15, 16)
                            for j in range(0, gray.shape[1]-15, 16)])
    
    # Гистограммные характеристики
    hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
    hist_norm = hist.ravel() / hist.sum()
    hist_entropy = entropy(hist_norm)
    
    # Текстурные характеристики
    lbp = local_binary_pattern(gray, 8, 1, method='uniform')
    lbp_hist = np.histogram(lbp, bins=np.arange(0, 11))[0]
    lbp_hist = lbp_hist.astype(float) / lbp_hist.sum()
    texture_entropy = entropy(lbp_hist)
    
    return {
        'sharpness': {
            'laplacian_var': laplacian_var,
            'gradient_mean': gradient_mean
        },
        'contrast': {
            'global': contrast,
            'local': local_contrast
        },
        'histogram': {
            'entropy': hist_entropy,
            'mean': np.mean(gray),
            'std': np.std(gray)
        },
        'texture': {
            'entropy': texture_entropy
        }
    }

def analyze_hard_cases(hard_cases_dir):
    hard_cases_dir = Path(hard_cases_dir)
    hard_cases = list(hard_cases_dir.glob('*.jpg'))
    
    problems = {
        'blur': [],      # Размытые изображения
        'dark': [],      # Темные изображения
        'bright': [],    # Засвеченные изображения
        'dirty': [],     # Грязные/стертые номера
        'angle': [],     # Сильный угол наклона
        'shadow': [],    # Тени на номере
        'partial': []    # Частично закрытые номера
    }
    
    # Сбор статистики по всем изображениям для нормализации
    all_metrics = []
    for img_path in hard_cases:
        img = cv2.imread(str(img_path))
        metrics = compute_image_quality_metrics(img)
        all_metrics.append(metrics)
    
    # Расчет пороговых значений
    thresholds = {
        'sharpness': {
            'laplacian_var': np.percentile([m['sharpness']['laplacian_var'] for m in all_metrics], 25),
            'gradient_mean': np.percentile([m['sharpness']['gradient_mean'] for m in all_metrics], 25)
        },
        'contrast': {
            'global': np.percentile([m['contrast']['global'] for m in all_metrics], 25),
            'local': np.percentile([m['contrast']['local'] for m in all_metrics], 25)
        },
        'brightness': {
            'low': np.percentile([m['histogram']['mean'] for m in all_metrics], 25),
            'high': np.percentile([m['histogram']['mean'] for m in all_metrics], 75)
        }
    }
    
    for img_path in hard_cases:
        img = cv2.imread(str(img_path))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        metrics = compute_image_quality_metrics(img)
        
        # 1. Анализ размытия (комплексный подход)
        is_blurry = (
            metrics['sharpness']['laplacian_var'] < thresholds['sharpness']['laplacian_var'] or
            metrics['sharpness']['gradient_mean'] < thresholds['sharpness']['gradient_mean']
        )
        if is_blurry:
            problems['blur'].append(img_path.name)
        
        # 2. Анализ грязи и стертости (улучшенный)
        local_entropy = metrics['texture']['entropy']
        local_contrast = metrics['contrast']['local']
        if local_entropy < np.mean([m['texture']['entropy'] for m in all_metrics]) * 0.7 or \
           local_contrast < thresholds['contrast']['local']:
            problems['dirty'].append(img_path.name)
        
        # 3. Анализ теней (улучшенный)
        v_channel = hsv[:, :, 2]
        thresh = threshold_otsu(v_channel)
        binary = v_channel < thresh
        shadow_ratio = np.sum(binary) / binary.size
        if shadow_ratio > 0.3:  # Более 30% темных областей
            problems['shadow'].append(img_path.name)
        
        # 4. Анализ яркого света/вспышек (улучшенный)
        if metrics['histogram']['mean'] > thresholds['brightness']['high']:
            v_channel_hist = np.histogram(v_channel, bins=256, range=(0, 256))[0]
            bright_ratio = np.sum(v_channel_hist[200:]) / np.sum(v_channel_hist)
            if bright_ratio > 0.1:  # Более 10% очень ярких пикселей
                problems['bright'].append(img_path.name)
        
        # 5. Анализ углов (улучшенный)
        # Используем комбинацию методов Хафа и градиентов
        edges = cv2.Canny(gray, 50, 150)
        lines = cv2.HoughLinesP(edges, 1, np.pi/180, 50, minLineLength=100, maxLineGap=10)
        
        if lines is not None:
            angles = []
            for line in lines:
                x1, y1, x2, y2 = line[0]
                angle = np.abs(np.degrees(np.arctan2(y2-y1, x2-x1)))
                angles.append(angle)
            
            # Анализируем распределение углов
            angle_hist = np.histogram(angles, bins=18, range=(0, 180))[0]
            angle_entropy = entropy(angle_hist / angle_hist.sum())
            if angle_entropy > 2.5 or np.std(angles) > 30:  # Высокая энтропия углов или большой разброс
                problems['angle'].append(img_path.name)
        
        # 6. Анализ частично закрытых номеров (улучшенный)
        # Используем анализ связных компонент и градиентов
        gradients = np.hypot(cv2.Sobel(gray, cv2.CV_64F, 1, 0), 
                           cv2.Sobel(gray, cv2.CV_64F, 0, 1))
        
        # Анализ распределения градиентов по областям
        h_blocks = np.array_split(gradients, 3, axis=1)
        v_blocks = np.array_split(gradients, 3, axis=0)
        
        h_means = [np.mean(block) for block in h_blocks]
        v_means = [np.mean(block) for block in v_blocks]
        
        # Проверяем неравномерность распределения градиентов
        h_variance = np.std(h_means) / np.mean(h_means)
        v_variance = np.std(v_means) / np.mean(v_means)
        
        if h_variance > 0.5 or v_variance > 0.5:  # Значительная неравномерность
            problems['partial'].append(img_path.name)
    
    # Анализ распределения проблем
    total_images = len(hard_cases)
    print("\nСтатистика распределения проблем:")
    print("-" * 50)
    for problem_type, images in problems.items():
        count = len(images)
        percentage = (count / total_images) * 100
        print(f"{problem_type}: {count} изображений ({percentage:.1f}%)")
    
    # Визуализация распределения проблем
    plt.figure(figsize=(12, 6))
    problem_counts = [len(cases) for cases in problems.values()]
    plt.bar(problems.keys(), problem_counts)
    plt.title('Распределение проблем в датасете')
    plt.xticks(rotation=45)
    plt.ylabel('Количество изображений')
    plt.tight_layout()
    plt.savefig('problem_distribution.png')
    plt.close()
    
    return problems

# Анализируем сложные случаи
problems = analyze_hard_cases('/kaggle/working/hard_cases_test/images')

# Подробный вывод результатов
print("\nАнализ сложных случаев:")
print("-" * 50)
for problem, cases in problems.items():
    print(f"\n{problem.upper()}: {len(cases)} случаев")
    if cases:
        print("Примеры:")
        for case in cases[:3]:
            print(f"  - {case}")


Статистика распределения проблем:
--------------------------------------------------
blur: 10 изображений (27.0%)
dark: 0 изображений (0.0%)
bright: 7 изображений (18.9%)
dirty: 9 изображений (24.3%)
angle: 6 изображений (16.2%)
shadow: 31 изображений (83.8%)
partial: 2 изображений (5.4%)

Анализ сложных случаев:
--------------------------------------------------

BLUR: 10 случаев
Примеры:
  - 1403707_af61f7dc10024f6cdc213d517f2f8940_photo.jpg
  - 1407817_4b6111bb4c6a7b488d1fbaae0b914661_photo.jpg
  - 1051833_fdc4fc24282f6b4052aac00a3ca0000f_photo.jpg

DARK: 0 случаев

BRIGHT: 7 случаев
Примеры:
  - 1049142_1a9445ffd3c4a2efac5bf0cf67cdf732_photo.jpg
  - 1407817_4b6111bb4c6a7b488d1fbaae0b914661_photo.jpg
  - 1400932_05d2e84faf6c4a0ac1b13bcfe963314d_photo.jpg

DIRTY: 9 случаев
Примеры:
  - 1407817_4b6111bb4c6a7b488d1fbaae0b914661_photo.jpg
  - 1051833_fdc4fc24282f6b4052aac00a3ca0000f_photo.jpg
  - 1399702_0f452608229dfa104d9473c520881a66_photo.jpg

ANGLE: 6 случаев
Примеры:
  - 1399702_

In [31]:
def validate_on_hard_cases(model, hard_cases_dir, epoch, problems):
    hard_cases_dir = Path(hard_cases_dir)
    hard_cases = list(hard_cases_dir.glob('*.jpg'))
    
    print(f"\nПроверка на сложных случаях (эпоха {epoch}):")
    
    save_dir = Path(f'runs/detect/hard_cases_epoch_{epoch}')
    save_dir.mkdir(parents=True, exist_ok=True)
    
    problem_stats = {
        'blur': {'total': len(problems['blur']), 'detected': 0},
        'dark': {'total': len(problems['dark']), 'detected': 0},
        'bright': {'total': len(problems['bright']), 'detected': 0},
        'dirty': {'total': len(problems['dirty']), 'detected': 0},
        'angle': {'total': len(problems['angle']), 'detected': 0},
        'shadow': {'total': len(problems['shadow']), 'detected': 0},
        'partial': {'total': len(problems.get('partial', [])), 'detected': 0}
    }
    
    for img_path in hard_cases:
        results = model.predict(str(img_path), save=False, verbose=False)[0]
        
        for problem_type, cases in problems.items():
            if img_path.name in cases:
                if len(results.boxes) > 0:
                    problem_stats[problem_type]['detected'] += 1
    
    print("\nРезультаты по типам проблем:")
    problem_detection_rates = {}
    for problem_type, stats in problem_stats.items():
        if stats['total'] > 0:
            detection_rate = (stats['detected'] / stats['total']) * 100
            problem_detection_rates[problem_type] = detection_rate
            print(f"{problem_type}: {stats['detected']}/{stats['total']} ({detection_rate:.1f}%)")
    
    return problem_detection_rates

def get_augmentation_config(problem_type, severity=1.0):
    """
    Возвращает конфигурацию аугментаций для конкретного типа проблемы
    severity: уровень интенсивности аугментаций (0.0 - 1.0)
    """
    import albumentations as A
    
    base_config = {
        'blur': A.Compose([
            A.OneOf([
                A.MotionBlur(blur_limit=7),
                A.GaussianBlur(blur_limit=7),
                A.MedianBlur(blur_limit=7),
            ], p=0.5 * severity),
            A.ImageCompression(quality_lower=60, quality_upper=100, p=0.3 * severity),
        ]),
        
        'dark': A.Compose([
            A.RandomBrightnessContrast(
                brightness_limit=(-0.3 * severity, 0.1),
                contrast_limit=(-0.2 * severity, 0.2),
                p=0.7 * severity
            ),
            A.HueSaturationValue(
                hue_shift_limit=5,
                sat_shift_limit=20,
                val_shift_limit=(-30 * severity, 10),
                p=0.5 * severity
            ),
        ]),
        
        'bright': A.Compose([
            A.RandomBrightnessContrast(
                brightness_limit=(0, 0.3 * severity),
                contrast_limit=(-0.2 * severity, 0.2),
                p=0.7 * severity
            ),
            A.HueSaturationValue(
                hue_shift_limit=5,
                sat_shift_limit=20,
                val_shift_limit=(10, 30 * severity),
                p=0.5 * severity
            ),
        ]),
        
        'dirty': A.Compose([
            A.CoarseDropout(
                max_holes=int(8 * severity),
                max_height=3,
                max_width=int(10 * severity),
                min_holes=1,
                p=0.5 * severity
            ),
            A.GridDistortion(num_steps=5, distort_limit=0.2 * severity, p=0.3 * severity),
        ]),
        
        'angle': A.Compose([
            A.ShiftScaleRotate(
                shift_limit=0.1 * severity,
                scale_limit=0.2 * severity,
                rotate_limit=30 * severity,
                p=0.7 * severity
            ),
            A.Perspective(scale=(0.05 * severity, 0.1 * severity), p=0.3 * severity),
        ]),
        
        'shadow': A.Compose([
            A.RandomShadow(
                num_shadows_lower=1,
                num_shadows_upper=int(3 * severity),
                shadow_dimension=4,
                shadow_roi=(0, 0.5, 1, 1),
                p=0.6 * severity
            ),
            A.HueSaturationValue(
                hue_shift_limit=5,
                sat_shift_limit=20,
                val_shift_limit=(-30 * severity, 10),
                p=0.4 * severity
            ),
        ]),
        
        'partial': A.Compose([
            A.CoarseDropout(
                max_holes=int(4 * severity),
                max_height=int(30 * severity),
                max_width=int(30 * severity),
                min_holes=1,
                p=0.5 * severity
            ),
            A.RandomCrop(
                height=int(256 * (1 - 0.3 * severity)),
                width=int(256 * (1 - 0.3 * severity)),
                p=0.3 * severity
            ),
        ])
    }
    
    return base_config.get(problem_type, None)

def adjust_augmentations(config, problem_detection_rates, epoch):
    print("\nКорректировка аугментаций:")
    
    for problem_type, rate in problem_detection_rates.items():
        if rate < 70:  # Если детекция ниже 70%
            # Рассчитываем интенсивность аугментаций на основе текущей производительности
            severity = min(1.0, (70 - rate) / 70 + 0.3)
            
            # Получаем специфичные аугментации для данного типа проблемы
            aug_config = get_augmentation_config(problem_type, severity)
            
            if problem_type == 'blur':
                config['sharpness'] = min(0.8, config.get('sharpness', 0.4) + 0.1 * severity)
                print(f"Увеличена резкость до {config['sharpness']}, severity={severity:.2f}")
                
            elif problem_type in ['dark', 'bright']:
                config['hsv_v'] = min(0.7, config.get('hsv_v', 0.4) + 0.1 * severity)
                config['contrast'] = min(0.6, config.get('contrast', 0.3) + 0.1 * severity)
                print(f"Усилены параметры яркости/контраста: hsv_v={config['hsv_v']}, contrast={config['contrast']}, severity={severity:.2f}")
                
            elif problem_type == 'angle':
                config['degrees'] = min(30.0, config.get('degrees', 20.0) + 2.0 * severity)
                config['perspective'] = min(0.002, config.get('perspective', 0.001) + 0.0002 * severity)
                print(f"Усилены угловые преобразования: degrees={config['degrees']}, perspective={config['perspective']}, severity={severity:.2f}")
                
            elif problem_type == 'shadow':
                config['hsv_v'] = min(0.8, config.get('hsv_v', 0.4) + 0.1 * severity)
                print(f"Усилены параметры для теней: hsv_v={config['hsv_v']}, severity={severity:.2f}")
                
            # Добавляем специфичные аугментации в конфигурацию
            if aug_config:
                config[f'{problem_type}_augmentations'] = aug_config
                print(f"Добавлены специальные аугментации для {problem_type}")
    
    with open(yaml_path, 'w') as f:
        yaml.dump(config, f, sort_keys=False)
    
    return config

In [32]:
# Обучение модели с адаптивными аугментациями
from ultralytics import YOLO

# Загружаем модель
model = YOLO("/kaggle/input/plate_detection/pytorch/default/1/best.pt")

# История обучения
history = {
    'problem_detection_rates': {},
    'best_rates': {}
}

for epoch in range(config['epochs']):
    print(f"\nЭпоха {epoch + 1}/{config['epochs']}")
    
    # Обучение одной эпохи
    model.train(
        data=yaml_path,
        epochs=1,
        resume=True if epoch > 0 else False,
        device=0
    )
    
    # Проверяем на сложных случаях
    problem_detection_rates = validate_on_hard_cases(
        model, 
        '/kaggle/working/hard_cases_test/images',
        epoch,
        problems
    )
    
    # Сохраняем историю
    history['problem_detection_rates'][epoch] = problem_detection_rates
    
    # Обновляем лучшие результаты
    for problem_type, rate in problem_detection_rates.items():
        if rate > history['best_rates'].get(problem_type, 0):
            history['best_rates'][problem_type] = rate
            print(f"Новый лучший результат для {problem_type}: {rate:.1f}%")
    
    # Корректируем аугментации каждую эпоху
    if epoch > 0:  # Начиная со второй эпохи
        config = adjust_augmentations(config, problem_detection_rates, epoch)
    
    # Визуализация прогресса
    plt.figure(figsize=(12, 6))
    for problem_type in problems.keys():
        rates = [history['problem_detection_rates'][e].get(problem_type, 0) 
                for e in range(epoch + 1)]
        plt.plot(rates, marker='o', label=problem_type)
    
    plt.title('Прогресс детекции по типам проблем')
    plt.xlabel('Эпоха')
    plt.ylabel('Процент детекций')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'problem_types_progress_epoch_{epoch+1}.png')
    plt.close()

# Итоговые результаты
print("\nИтоговые результаты:")
for problem_type, best_rate in history['best_rates'].items():
    print(f"\n{problem_type}:")
    print(f"  Начальный процент детекций: {history['problem_detection_rates'][0][problem_type]:.1f}%")
    print(f"  Лучший процент детекций: {best_rate:.1f}%")
    print(f"  Конечный процент детекций: {history['problem_detection_rates'][config['epochs']-1][problem_type]:.1f}%")


Эпоха 1/25
Ultralytics 8.3.192 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/Car_plate_detecting/Config/finetune.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/kaggle/input/plate_detection/pytorch/default/1/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train6, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, o

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       2563       2678      0.976      0.964      0.986      0.822

1 epochs completed in 0.432 hours.
Optimizer stripped from runs/detect/train6/weights/last.pt, 51.6MB
Optimizer stripped from runs/detect/train6/weights/best.pt, 51.6MB

Validating runs/detect/train6/weights/best.pt...
Ultralytics 8.3.192 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv9c summary (fused): 156 layers, 25,320,019 parameters, 0 gradients, 102.3 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 81/81 1.8it/s 44.0s0.7ss


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       2563       2678      0.974      0.965      0.986      0.822
Speed: 0.1ms preprocess, 13.3ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/train6

Проверка на сложных случаях (эпоха 0):

Результаты по типам проблем:
blur: 3/10 (30.0%)
bright: 1/7 (14.3%)
dirty: 1/9 (11.1%)
angle: 2/6 (33.3%)
shadow: 10/31 (32.3%)
partial: 1/2 (50.0%)
Новый лучший результат для blur: 30.0%
Новый лучший результат для bright: 14.3%
Новый лучший результат для dirty: 11.1%
Новый лучший результат для angle: 33.3%
Новый лучший результат для shadow: 32.3%
Новый лучший результат для partial: 50.0%

Эпоха 2/25
Ultralytics 8.3.192 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/

RuntimeError: Dataset '/kaggle/working/Car_plate_detecting/config_dataset.yaml' error ❌ Dataset '/kaggle/working/Car_plate_detecting/config_dataset.yaml' images not found, missing path '/kaggle/working/dataset/val'
Note dataset download directory is '/kaggle/working/datasets'. You can update this in '/root/.config/Ultralytics/settings.json'